In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=6

import struct
import numpy as np
import pandas as pd
import networkx as nx
import scipy
from scipy.stats import entropy
from scipy.sparse import csr_matrix 
from tqdm import tqdm
from matplotlib import pyplot as plt
from collections import defaultdict
import pickle
import argparse
import sys

sys.path.insert(1, os.path.join(sys.path[0], '..'))

from functions import absorbing_markov, reduce_Q

parser = argparse.ArgumentParser(description='Run demo')
parser.add_argument('--nx_gpickle_file', type=str, default="", help='networkx pickle file to process')

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
import datashader

In [ ]:
import datashader as ds, pandas as pd, colorcet

In [ ]:
to_display = [
    ['graph_pk_usdc.gpickle', 'usdc_pk'],
    ['graph_pk_dai.gpickle', 'dai_pk'],
    ['graph_pk_bitcoin.gpickle', 'bitcoin_pk'],
    ['graph_pk_ethereum.gpickle', 'ethereum_pk'],
    ['graph_pk_zcash_spb.gpickle', 'zcash_pk'],
]

In [ ]:
points_coll = {}

In [ ]:
import math

In [ ]:
for gp, fold in to_display:
    args = parser.parse_args(('--nx_gpickle_file graph/'+gp).split(' '))

    np.random.seed(123)

    G = nx.read_gpickle(args.nx_gpickle_file)
    zeroSats=0
    toBeDeletedEdges = []
    for e in tqdm(G.edges.data("weight")):
        if e[2]==0:
            zeroSats+=1
            toBeDeletedEdges.append(e)
    G.remove_edges_from(toBeDeletedEdges)
    from scipy.sparse import identity
    from scipy.sparse.linalg import inv

    I, Q, R, node_recode, absorber_recode = absorbing_markov(G)
    i=1
    Q_slice, R_slice, slice_size, conv_lim, verbose = Q, np.ones_like(R[:, i:i+1].todense()), 1, 1e-6, True
    qkr = R_slice
    coll = qkr.copy()
    for i in tqdm(range(10000)):
        qkr = Q_slice*qkr
        coll += qkr
    import pickle
    with open(fold+'/recode', 'rb') as handle:
        recode = pickle.load(handle)
    if fold == "zcash_pk_comparison":
        entropies = np.load(fold+'/entropies_modified.npy')
    else:
        entropies = np.load(fold+'/entropies.npy')
    unspent = np.load(fold+'/unspent.npy')
    step_ixs = [node_recode[n] for n in recode.keys()]
    entropy_ixy = [recode[n] for n in recode.keys()]
    points = pd.DataFrame({'steps': np.array(coll).squeeze()[np.array(step_ixs)], 'entropies': entropies[np.array(entropy_ixy)]})
    points_coll[fold] = points
    cvs = ds.Canvas(plot_width=500, plot_height=500)
    agg = cvs.points(points, 'steps', 'entropies')
    img = ds.tf.shade(agg, cmap=colorcet.fire)
    display(HTML("<h3>"+fold+"</h3>"+img._repr_html_()))

In [ ]:
for k, p in points_coll.items():
    p.to_csv('expected_steps/'+k+'.csv.gz')

In [ ]:
points_coll2 = points_coll

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import math
import colorcet
import numpy as np

In [ ]:
to_display = [
    ['graph_pk_usdc.gpickle', 'usdc_pk'],
    ['graph_pk_dai.gpickle', 'dai_pk'],
    ['graph_pk_bitcoin.gpickle', 'bitcoin_pk_comparison_a100'],
    ['graph_pk_ethereum.gpickle', 'ethereum_pk_comparison'],
    ['graph_pk_zcash_spb.gpickle', 'zcash_pk_comparison'],
]
points_coll={}

In [ ]:
for k,v in to_display:
    points_coll[v] = pd.read_csv('expected_steps/'+v+'.csv.gz')
    points_coll[v]['entropies'] *= 1/math.log(2)

In [ ]:
from datashader import transfer_functions as tf
from datashader.colors import inferno, viridis
from matplotlib.colors import LinearSegmentedColormap
cm = LinearSegmentedColormap.from_list(colors=colorcet.fire, name="asd")
def plot_points(points, name, max_x=1000, max_y=None, x_step=200, bins=300, figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.title(name)
    plt.xlabel('Steps')
    plt.ylabel('Untracability')
    points_ = points[points['steps']<max_x]
    if max_y:
        points_ = points_[points_['entropies']<max_y]
    hist = np.histogram2d(*points_.values.T, bins=bins)
    data = tf.eq_hist(hist[0].T)[0]
    yticks = np.arange(0, hist[2].max()+1)
    yticks_loc = yticks/hist[2].max()*bins
    xticks = np.arange(0,max_x+1, x_step)
    xticks_loc = xticks/hist[1].max()*bins
    plt.xticks(xticks_loc, xticks)
    plt.yticks(yticks_loc, yticks)
    plt.imshow(
        data,
        cmap=cm,
        origin="lower",
        #alpha=data==0
    )

In [ ]:
cm = LinearSegmentedColormap.from_list(colors=colorcet.fire[::-1], name="asd")

In [ ]:
plot_points(points_coll['usdc_pk'][['steps','entropies']], 'USD Coin', max_x=25, max_y=4.5*(1/math.log(2)), x_step=5)
plt.savefig("expected_steps/usdc.pdf")

In [ ]:
plot_points(points_coll['dai_pk'][['steps','entropies']], 'DAI', max_x=100, max_y=4.5*(1/math.log(2)), x_step=20)
plt.savefig("expected_steps/dai.pdf")

In [ ]:
plot_points(points_coll['bitcoin_pk_comparison_a100'][['steps','entropies']], 'Bitcoin', max_x=200, max_y=11*(1/math.log(2)), x_step=50)
plt.savefig("expected_steps/bitcoin.pdf")

In [ ]:
plot_points(points_coll['ethereum_pk_comparison'][['steps','entropies']], 'Ethereum', max_x=15, max_y=7*(1/math.log(2)), x_step=3)
plt.savefig("expected_steps/ethereum.pdf")

In [ ]:
plot_points(points_coll['zcash_pk_comparison'][['steps','entropies']], 'Zcash', max_x=1000, max_y=11*(1/math.log(2)), x_step=200)
plt.savefig("expected_steps/zcash.pdf")